<!-- TABS -->
# Text Vector Search

<!-- TABS -->
## Connect to superduper

:::note
Note that this is only relevant if you are running superduper in development mode.
Otherwise refer to "Configuring your production system".
:::

In [ ]:
from superduper import superduper

db = superduper('mongomock:///test_db')

<!-- TABS -->
## Get useful sample data

In [ ]:
# <tab: Text>
!curl -O https://superduper-public-demo.s3.amazonaws.com/text.json
import json

with open('text.json', 'r') as f:
    data = json.load(f)

In [ ]:
# <tab: PDF>
!curl -O https://superduper-public-demo.s3.amazonaws.com/pdfs.zip && unzip -o pdfs.zip
import os

data = [f'pdfs/{x}' for x in os.listdir('./pdfs') if x.endswith('.pdf')]

In [ ]:
datas = [{'x': d} for d in data]

<!-- TABS -->
## Create datatype

SuperduperDB supports automatic data conversion, so users don’t need to worry about the compatibility of different data formats (`PIL.Image`, `numpy.array`, `pandas.DataFrame`, etc.) with the database.

It also supports custom data conversion methods for transforming data, such as defining the following Datatype.

In [ ]:
# <tab: Text>
datatype = 'str'

In [ ]:
# <tab: PDF>
from superduper import DataType

# By creating a datatype and setting its encodable attribute to “file” for saving PDF files, 
# all datatypes encoded as “file” will have their corresponding files uploaded to the artifact store. 
# References will be recorded in the database, and the files will be downloaded locally when needed. 

datatype = DataType('pdf', encodable='file')

<!-- TABS -->
## Setup tables or collections

In [ ]:
from superduper.components.table import Table
from superduper import Schema

schema = Schema(identifier="schema", fields={"x": datatype})
table_or_collection = Table("documents", schema=schema)
db.apply(table_or_collection)

Inserting data, all fields will be matched with the schema for data conversion.

In [ ]:
db['documents'].insert(datas).execute()
select = db['documents'].select()

<!-- TABS -->
## Apply a chunker for search

:::note
Note that applying a chunker is ***not*** mandatory for search.
If your data is already chunked (e.g. short text snippets or audio) or if you
are searching through something like images, which can't be chunked, then this
won't be necessary.
:::

In [ ]:
# <tab: Text>
from superduper import model

CHUNK_SIZE = 200

@model(flatten=True, model_update_kwargs={'document_embedded': False})
def chunker(text):
    text = text.split()
    chunks = [' '.join(text[i:i + CHUNK_SIZE]) for i in range(0, len(text), CHUNK_SIZE)]
    return chunks

In [ ]:
# <tab: PDF>
!pip install -q "unstructured[pdf]"
from superduper import model
from unstructured.partition.pdf import partition_pdf

CHUNK_SIZE = 500

@model(flatten=True)
def chunker(pdf_file):
    elements = partition_pdf(pdf_file)
    text = '\n'.join([e.text for e in elements])
    chunks = [text[i:i + CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]
    return chunks

Now we apply this chunker to the data by wrapping the chunker in `Listener`:

In [ ]:
from superduper import Listener

upstream_listener = Listener(
    model=chunker,
    select=select,
    key='x',
    uuid="chunk",
)

db.apply(upstream_listener)

## Select outputs of upstream listener

:::note
This is useful if you have performed a first step, such as pre-computing 
features, or chunking your data. You can use this query to 
operate on those outputs.
:::

In [ ]:
indexing_key = upstream_listener.outputs_key
select = upstream_listener.outputs_select

<!-- TABS -->
## Build text embedding model

In [ ]:
# <tab: OpenAI>
!pip install openai
from superduper_openai import OpenAIEmbedding

embedding_model = OpenAIEmbedding(identifier='text-embedding-ada-002')

In [ ]:
# <tab: JinaAI>
import os
from superduper_jina import JinaEmbedding

os.environ["JINA_API_KEY"] = "jina_xxxx"
 
# define the model
embedding_model = JinaEmbedding(identifier='jina-embeddings-v2-base-en')

In [ ]:
# <tab: Sentence-Transformers>
!pip install sentence-transformers
from superduper import vector
import sentence_transformers
from superduper_sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(
    identifier="embedding",
    object=sentence_transformers.SentenceTransformer("BAAI/bge-small-en"),
    datatype=vector(shape=(1024,)),
    postprocess=lambda x: x.tolist(),
    predict_kwargs={"show_progress_bar": True},
)

In [ ]:
print(len(embedding_model.predict("What is superduper")))

## Create vector-index

In [ ]:
vector_index_name = 'my-vector-index'

In [ ]:
from superduper import VectorIndex, Listener

jobs, _ = db.apply(
    VectorIndex(
        vector_index_name,
        indexing_listener=Listener(
            key=indexing_key,      # the `Document` key `model` should ingest to create embedding
            select=select,       # a `Select` query telling which data to search over
            model=embedding_model,         # a `_Predictor` how to convert data to embeddings
        )
    )
)

In [ ]:
query_table_or_collection = select.table_or_collection

## Perform a vector search

In [ ]:
from superduper import Document
# Perform the vector search based on the query
item = Document({indexing_key: "Tell me about the superduper"})

In [ ]:
select = query_table_or_collection.like(item, vector_index=vector_index_name, n=5).select()
results = list(db.execute(select))

In [ ]:
for result in results:
    print("\n", '-' * 20, '\n')
    print(Document(result.unpack())[indexing_key])